# 建模流程Check list

## 问题定义（打标）
1. 分析业务流程并挑选关键环节作为切入点，挑选原则
    - 具有业务意义，*如：KPI指标、或转化较低的环节等*
    > 例如：以识别客户资金需求为目标，首先对所能获取的数据有一个全面的了解后，可以使用招联新增借款，人行新增借款和联通新增借贷类app等数据作为判断资金需求的依据
    - 有可靠数据支持， *如：无法获取客户浏览短信的数据，则不能将浏览与否作为打标依据，可考虑使用登录行为替代*
    > 例如：当使用联通数据时，数据波动性较强，某时点之前的营销数据均会被删除，某时点之后的互联网标签数据口径不一致等，所以要求在建模前对可用数据进行验证，根据数据情况确定观察期与表现期，同时确保训练使用的数据在模型上线之后可以获取且口径一致
   
   
2. 将业务目标抽象为具体的问题
    - 分类
    - 聚类
    - 回归等

## SQL（数据粗加工）

1. 样本选取
    - 根据业务目标确定目标客群
    - 根据目标客群数据量判断是否需要抽样
    > 注意结果导向，尽可能保证识别出来的客群能够满足目标基本要求，防止模型测试时由于不可控因素导致转化下降，如识别借款需求是为了新增借款，但新增借款的前提是能通过风控挡板，所以当前具有高风险特征（例如：高共债）的客户就不应该入模，这部分客户的模型评分就会为null，为避免使用者疑惑，可备注其无评分原因，具体场景的入模样本需与业务方沟通


2. 特征粗筛
    - 为避免数据量过大，需根据业务目标挑选相关分类的特征进行加工

    > 注意：观察期（特征加工）与表现期（打标）使用的数据不能有时间上的重叠


3. **Code check list**
   - 没有按照通用的格式对齐各字段，字段类型以及字段说明等
   - 没有在`case when`语句最后添加上`else null end`或者`else 0 end`
   - 在impala查询语句中，不能在`left join`条件里面加上不等式，如：>=，<=等
   - sql语句中存在大量不必要的`select distinct`语句
   - 存在类似于`select *`，`select count(*)`的语句，不符合规范，尽量只选取需要的列
   - 注意少用子查询，可用`with subquerry as table name`语法代替提高并行度，对于反复使用的逻辑建立中间表提高代码复用率
   - 注意使用**谓词下推**提高运行效率，将过滤表达式尽可能移动至靠近数据源的位置，以使真正执行时能直接跳过无关的数据
   - 加工表时注意加上数据加工日期`etl_dt`
   - 连接表时注意各个子表主键是否唯一


## Python（数据细加工&建模）

1. 数据探索
   - 特征中能否挖掘出有效的专家规则？
      - 若有则可以直接通过部署专家规则的形式解决问题，节省开发时间
      - 方法：绘制分布图、相关性分析等
      > 此过程中需重点关注与目标计算相关的变量（如模型目标是识别借贷需求，那么当前是否有贷有很大可能与目标强相关），若有强相关变量则考虑是否分卡建模，因为强相关变量往往可以将样本切分为具有不同性质的客群，这些性质有些难以通过特征捕捉，杂糅在一个模型中就容易造成误差
   - 样本是否平衡？
      - 若极不平衡(通常正负样本比例<1/10)，则需进行样本平衡
      - 方法：过（欠）采样、人工合成样本、迁移学习等
      
      
2. 特征加工（根据业务、数据以及所选模型情况决定）
   - 数据类型转化
   - 空缺值&异常值处理
   - 标准化、归一化（针对特定模型，如：广义线性模型）
   - 分桶
   - 特征交叉
   - 降维
   
   
3. 训练、验证和测试样本划分
   - 训练：在部分样本中学习数据的概率分布
   - 验证：用训练数据之外的数据调整模型超参，平衡偏差和方差，提高模型泛化性能
   - 测试：用训练及验证之外的数据对模型性能进行离线测试

> 注意：
> 1. 训练、验证和测试数据集尽量不要产生人与时间的交叉
> 2. 为避免由于人为数据集划分不均导致的测试效果不客观，可以使用交叉验证

4. 模型训练&评估
   - 广义线性模型
   - 树模型
   - 集成模型
   - 神经网络
   - ...
   - 混淆矩阵
   - tpr = tp/(tp+fn)
   - fpr = fp/(tn+fp)
   - ks = max(tpr-fpr)
   - accuracy = (tp+tn)/(tp+tn+fp+fn)
   - precision = tp/(tp+fp)
   - recall = tp/(tp+fn)
   - lift = 分段响应率/总体响应率（营销关注）
   
   
5. 效果&效能提升
   - 特征选择：根据模型初步训练结果筛选重要特征
   - 模型选择：根据模型初步训练结果筛选效果好的模型
   - 模型融合：将多个模型结果进行融合
   - Refit：将验证与测试数据集用于训练
   
> 注意：模型融合中子模型最好分属于不同分类

6. Code check list
   - 代码注释是否清晰，命名是否规范
      - 变量&函数命名&模块&包命名: 小写字母，单词之间用下划线分割，用简短的单词概括变量含义，如：`loan_amt, get_loan_amt(), pandas.py`
      - 类命名：单词首字母大写，如：`TreeNode`
   - 变量统一定义，是否存在硬编码现象
   - 重复的操作是否抽象成模块、类或函数
   - 循环操作是否可用矩阵操作替代，简单操作可直接通过numpy的矩阵运算实现，复杂操作可以使用pandas的apply函数


## 线上效果验证 
在模型开发完成后需先进行线上效果测试，测试效果满足上线要求后再进行部署，一般的效果验证流程如下：

- Step1. 咨询业务同事近期的活动计划，挑选与模型目标相吻合的活动，主要获取两部分信息：**客群**和**评价指标**
> 例如：运营同事近期要对注册未授信客群进行营销打捞，目的为提升填写响应率（营销后来填写申请人数/触达人数）

- Step2. 结合活动计划设计模型测试方案，以**控制变量法**为原则，围绕模型的功能点对客群进行分组对比验证
> 例如：打捞模型具有预测填写响应分和推荐营销短信话术两个功能点，
> * 为了验证响应分的排序性，根据模型分数将总客群分为高分客群、中分客群和低分客群3组，采用相同的话术进行营销，观察响应情况
> * 为了验证推荐话术的准确性，将相同分数段的客户分为两组，分别采用模型推荐话术以及专家规则选定的话术进行触达，观察响应情况
    
- Step3. 对各组客群执行相应策略，待表现期满计算Step1中的评价指标衡量模型效果
> 不同业务衡量模型效果的方法略有不同，达标的标准也有差异，具体需与业务人员商定
> 例如：营销场景主要使用`lift`作为衡量模型效果的指标，在打捞场景下若行动组高分客群（**约前20%**）的填写响应率/整体填写响应率**>2.5**则认定模型效果较为显著，可以部署上线

## 例子

### 函数注释

In [ ]:
# 函数注释:"""功能概述， 空格， 参数说明， 返回值说明"""
def get_fee(loan_bal, fee_rate):
    """Calculate fee amount by multiplying loan_bal and fee_rate 
    
    Args:
        loan_bal: numeric, value balance of the loan
        fee_rate: float, fee rate of the loan
        
    Returns:
        fee amount
    """
    return loan_bal * fee_rate

### 矩阵操作

In [ ]:
# numpy 矩阵操作
import numpy as np
import time

# 生成测试数据
data_len, data_wid = 100, 100
arr1 = np.random.rand(data_len, data_wid)
arr2 = np.random.rand(data_len, data_wid)
arr3 = np.random.rand(data_len, data_wid)
print("数据量：{}".format(data_len, data_wid))

# 测试1：循环运算
start_tm1 = time.time()
print("测试1进行中……")
for i in range(arr1.shape[0]):
    for j in range(arr1.shape[1]):
        arr3[i,j] = arr1[i,j] + arr2[i,j]
end_tm1 = time.time()

# 测试2：矩阵运算
start_tm2 = time.time()
print("测试2进行中……")
arr3 = arr1 + arr2
end_tm2 = time.time()

# 测试结果
test1_cost, test2_cost = end_tm1 - start_tm1, end_tm2 - start_tm2
print("测试1用时是测试2的{:.2f}倍".format(test1_cost/test2_cost))

In [ ]:
# pandas 矩阵操作
import pandas as pd
from datetime import datetime

raw_data = pd.read_csv('../input/groceries-dataset/Groceries_dataset.csv')
print("数据量：{}".format(raw_data.shape))
print(raw_data.head())


# 定义函数
def date_to_days(date_str, date_format='%d-%m-%Y'):
    '''Convert date string to time type
    
    Args:
        date_str: string, date
        date_format: string, format of date(the first arguement)
    
    Returns:
        time type of the date_str
    '''
    
    date = datetime.strptime(date_str, date_format)
    days = (datetime.now() - date).days
    return days

# 测试1：循环运算
start_tm1 = time.time()
print("测试1进行中……")
test_df = pd.DataFrame()
for i in range(len(raw_data)):
    test_df.loc[i, 'days'] = date_to_days(raw_data.loc[i, 'Date'])

end_tm1 = time.time()

# 测试2：矩阵运算
start_tm2 = time.time()
print("测试2进行中……")
test_df = pd.DataFrame()
test_df['days']  = raw_data['Date'].apply(date_to_days)
test_df.head()
end_tm2 = time.time()


# 测试结果
test1_cost, test2_cost = end_tm1 - start_tm1, end_tm2 - start_tm2
print("测试1用时是测试2的{:.2f}倍".format(test1_cost/test2_cost))

## 欢迎补充！